### Install dependencies and Mount Gdrive

In [ ]:
#Install libretorrent for python
!apt install python3-libtorrent
!pip3 install lbry-libtorrent

#mount Gdrive
from google.colab import drive
drive.mount('/content/drive')

### Downloader

In [ ]:
import libtorrent as lt
import time, os, shutil, re, string

#torrent download path
TDPATH = "Work_space/DTORRENT/"
#use to remove all unwanted char from strings
chars = re.escape(string.punctuation)

#greeter
print("##############################################################")
print("##                                                          ##")
print("##  TTTTTTTTTTTT  TTTTTTTTTTTT  GGGGGGGGGGGG  DDDDDDDDDD    ##")
print("##       TT            TT       GG            DD        DD  ##")
print("##       TT            TT       GG  GGGGGGGG  DD        DD  ##")
print("##       TT            TT       GG        GG  DD        DD  ##")
print("##       TT            TT       GG        GG  DD        DD  ##")
print("##       TT            TT       GG        GG  DD        DD  ##")
print("##       TT            TT       GGGGGGGGGGGG  DDDDDDDDDD    ##")
print("##                                                          ##")
print("##############################################################")

print("Waiting for your input... :)")

#downloader
link = input("Paste a magnet link and hit enter:")
SES = lt.session()
SES.listen_on(6881, 6891)
params = {
    'save_path': TDPATH,
    'storage_mode': lt.storage_mode_t(2),
    }
handle = lt.add_magnet_uri(SES, link, params)
SES.start_dht()

print("downloading metadata...")
while (not handle.has_metadata()):
    time.sleep(1)
pName = str(handle.status().name)
print(f"got metadata, {pName} downloading started...")
while handle.status().state != lt.torrent_status.seeding:
    s = handle.status()
    state_str = ['queued', 'checking', 'downloading metadata',
                 'downloading', 'finished', 'seeding', 'allocating']
    print('%.2f%% complete (down: %.1f kb/s up: %.1f kB/s peers: %d) %s ' %
          (s.progress * 100, s.download_rate / 1000, s.upload_rate / 1000,
           s.num_peers, state_str[s.state]))
    time.sleep(5)

#archiving moving files
name = str(re.sub(r'['+chars+']', '',(pName.encode("ascii", "ignore")).decode())).replace(" ", "_")
print(f"{name} Downloading completed...\nAdding files to an archive...")

time.sleep(5)
shutil.move(f"Work_space/DTORRENT/{pName}", f"Work_space/DTORRENT/{name}")
shutil.make_archive(f"Work_space/CTORRENT/{name}", "tar", f"Work_space/DTORRENT/{name}")
time.sleep(5)
print("Moving archive to Gdrive...")
os.system(f"mv Work_space/CTORRENT/{name}.tar drive/MyDrive/{name}.tar")
print("Cleaning the disk...")
shutil.rmtree(f"Work_space/DTORRENT/{name}", ignore_errors=True)
time.sleep(5)

print("All done... Check you Gdrive... :)")
print("If you are done for today, Please make sure to disconnect and delete runtime before leaving... :*")

### Something went wrong in the above code, Run this :)

In [ ]:
!rm -r Work_space/CTORRENT/*
!rm -r Work_space/DTORRENT/*

### Please make sure to disconnect and delete runtime before you leave here